In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import time


In [2]:
def getInput():
    stateList=[]
    region = input("Enter region: ") 
    states = input(('Enter list of states in the region as a comma separated string, e.g for  Midwest region enter  Ohio,'
                   'Indiana, Michigan, Iowa,North Dakota, South Dakota, Nebraska, Kansas, Missouri, Minnesota, Wisconsin, Illinois')) 

    stateList = states.split(",")
    return {"region": region,"states": stateList}

In [3]:
def visitState(statename):
    browser.is_element_present_by_id("sc.keyword", wait_time=1)
    browser.find_by_id('sc.keyword').fill('Data Scientist')
    browser.find_by_id('sc.location').fill(statename)
    browser.find_by_id('HeroSearchButton').click()

In [4]:
def addAgeFilter():
    while False:
        browser.is_element_present_by_id("filter_fromAge", wait_time=2)
    div = browser.find_by_id("filter_fromAge")
    div.click()
    while False:
        browser.is_element_present_by_id("PrimaryDropdown", wait_time=2)
    divFilter = browser.find_by_id("PrimaryDropdown")
    divFilter.find_by_text('Last Month').click()

In [5]:
def searchJobs():
    # instantiate the list
    jobs = []

    #complete the notebook run no matter what
    try:
        while True:

            #trigger popup and close it
            try:
                trigger_and_close_popup(browser)
            except Exception as e:
                pass

            #close any possible popups with a Close button
            try:
                browser.driver.find_element_by_partial_link_text('Close').click()
                time.sleep(1)
            except Exception as e:
                pass

            #more popups
            try:
                browser.driver.find_element_by_class_name('ModalStyle__xBtn___29PT9').click()
                time.sleep(1)
            except Exception as e:
                pass

            html = browser.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')


            article = soup.find('article', id='MainCol').ul
            job_list = article.find_all('li')

            for job  in job_list:
                #title
                title = job['data-normalize-job-title'].strip()
                #job location
                location  = job['data-job-loc'].strip()
                #employer
                employer = job.find('div', class_='jobEmpolyerName').text

                #job details
                dataId = job['data-id']
                Id = 'JobDesc' + dataId
                xpath = f"//li[@data-id='{dataId}']"
                try:
                    soup = soup_for_click(xpath)
                    job_desc = soup.find('div', id = Id, class_="jobDesc").text
                except Exception as e:
                    job_desc = "" 

                #review count
                xpath= '//*[@id="Details"]/div[1]/div[1]/div/div/div[5]'
                try:
                    soup = soup_for_click(xpath)
                    reviews= soup.select("li.empReview")
                    review_count = len(reviews)
                except Exception as e:
                    review_count = 0

                #create the job dictoanry object and add it to the list
                job_dict = {"position": title,"company": employer, "description": job_desc, "reviews": review_count, \
                            "location":location}

                jobs.append(job_dict)

            try:
                url=browser.url
                browser.find_by_css('li.next').click()
                time.sleep(1)
                if (url == browser.url):
                        break
            except NoSuchElementException:
                print("No such element exception.")
                break

        print("Scraping complete.")

    # exit gracefully, close the browser, log the exception 
    except Exception as e:
        print("Scraping interrupted.")
    return jobs

In [6]:
def soup_for_click(xpath):
    browser.driver.find_element_by_xpath(xpath).click()
    time.sleep(1)
    html = browser.driver.page_source
    return BeautifulSoup(html, 'html.parser')

In [7]:
def trigger_and_close_popup(browser):
    browser.is_element_present_by_id("MainCol", wait_time=1)
    xpath= '//*[@id="MainCol"]/div/ul/li[1]'
    browser.driver.find_element_by_xpath(xpath).click()
    time.sleep(1)

    xpath ='//*[@id="JAModal"]/div/div/div[2]/div/div[1]/div/span'
    browser.driver.find_element_by_xpath(xpath).click()
    time.sleep(1)


In [8]:
# region_states = getInput()
# region_states


Enter region: West
Enter list of states in the region as a comma separated string, e.g for  Midwest region enter  Ohio,Indiana, Michigan, Iowa,North Dakota, South Dakota, Nebraska, Kansas, Missouri, Minnesota, Wisconsin, IllinoisCalifornia,Nevada,Utah,Montana,Colorado,Wyoming,Idaho,Oregon,Washington State


{'region': 'West',
 'states': ['California',
  'Nevada',
  'Utah',
  'Montana',
  'Colorado',
  'Wyoming',
  'Idaho',
  'Oregon',
  'Washington State']}

In [9]:
# region = region_states['region']
# stateList = region_states['states']

region='set your region here once'

In [60]:
# exe_path = 'c:/Users/indup/bin/chromedriver'
exe_path='set your chrome path here'
executable_path = {'executable_path': exe_path }
browser = Browser('chrome', **executable_path)


In [62]:
#spider for url
url="https://www.glassdoor.com"
browser.visit(url)

In [47]:
# select google authorization
while False:
    browser.is_element_present_by_css("button.google")
browser.find_by_css("button.google").click()

In [48]:
# switch to oauth window
window1 = browser.driver.window_handles[1]
browser.driver.switch_to.window(window1)
username = 'set your gmail id here'
browser.find_by_id("identifierId").fill(username)
browser.find_by_id("identifierNext").click()


In [49]:
while False:
    browser.is_element_present_by_name("password",wait_time=4)
pwd= 'your password here'
browser.find_by_name("password").fill(pwd)
browser.find_by_id("passwordNext").click()


In [50]:
window1 = browser.driver.window_handles[0]
browser.driver.switch_to.window(window1)

In [51]:
try:
    browser.find_by_id("a").click()
except:
    pass

In [59]:
# while len(stateList) > 0 :
#     stateName = stateList.pop().strip().lower()
# stateName = 'Oregon'
stateName='set your state here everytime'
visitState(stateName)
addAgeFilter()
jobs = searchJobs()
glassdoor_df = pd.DataFrame(jobs)
glassdoor_df["Region"] = region
glassdoor_df.to_excel(f"../../Transform/Resources/Glassdoor/{stateName}_glassdoor.xlsx", index=False)


Scraping complete.
